In [7]:
import pandas as pd
import nameparser
import json
import requests
import numpy as np

pro_publica_api_key = "PguHImFJakafbsBpZ2ed21A1HIgvXKiM7tg0BqI0 "
headers = {'X-API-Key': pro_publica_api_key}

In [8]:

members = pd.read_csv('../prod_data/transactions.csv')

df = members[['id', 'congress']].drop_duplicates()

df

,id,congress
0,L000590,116
31,L000579,116
34,B001274,116
36,W000804,116
44,D000617,116
...,...,...
24000,S001203,118
24021,M000355,118
24024,S001198,118
24030,B001236,118


In [9]:
new_ids = pd.read_csv('../prod_data/new_ids.csv')[['id', 'congress']]

new_ids

,id,congress
0,C000174,116
1,M000934,116
2,C001047,116
3,C001035,116
4,P000595,116
...,...,...
98,S001203,118
99,M000355,118
100,S001198,118
101,B001236,118


In [10]:
current_statements_prod = pd.read_csv('../prod_data/member_statements.csv')
current_statements_prod

,url,date,title,type,member_id,congress
0,https://susielee.house.gov/media/press-release...,2020-12-03,Rep. Lee Announces Nearly $1.4 Million Grant t...,Press Release,L000590,116
1,https://susielee.house.gov/media/press-release...,2020-12-01,"Problem Solvers Caucus, Senate Partners Unveil...",Press Release,L000590,116
2,https://susielee.house.gov/media/press-release...,2020-11-20,Rep. Susie Lee Helps Pass National Apprentices...,Press Release,L000590,116
3,https://susielee.house.gov/media/press-release...,2020-11-18,Rep. Susie Lee Elected to Serve as Regional Re...,Press Release,L000590,116
4,https://susielee.house.gov/media/press-release...,2020-12-27,"Rep. Lee Statement, President Trump Finally Si...",Press Release,L000590,116
...,...,...,...,...,...,...
45788,https://evans.house.gov/media-center/press-rel...,2023-02-28,Evans Introduces Resolution Pushing to End Ine...,Press Release,E000296,118
45789,https://evans.house.gov/media-center/press-rel...,2023-02-22,Letter of Support - Reconnecting Our Chinatown...,Press Release,E000296,118
45790,https://evans.house.gov/media-center/press-rel...,2023-02-09,Evans Makes Super Bowl Bet with Kansas City Co...,Press Release,E000296,118
45791,https://evans.house.gov/media-center/press-rel...,2023-02-06,Evans Welcomes a Philly Housing Leader to Stat...,Press Release,E000296,118


In [11]:
current_statements_cache = pd.read_csv('member_statements.csv')

current_statements_cache

,url,date,title,type,member_id,congress
0,https://www.carper.senate.gov/public/index.cfm...,2020-12-31,Carper: EPAs Censoring Science Rule is the Tru...,Press Release,C000174,116
1,https://www.carper.senate.gov/public/index.cfm...,2020-12-30,Carper Celebrates the 50th Anniversary of the ...,Press Release,C000174,116
2,https://www.carper.senate.gov/public/index.cfm...,2020-12-22,Delaware Delegation Celebrates Major Wins for ...,Press Release,C000174,116
3,https://www.carper.senate.gov/public/index.cfm...,2020-12-22,Carper Highlights Significant Climate Solution...,Press Release,C000174,116
4,https://www.carper.senate.gov/public/index.cfm...,2020-12-21,Carper Cheers Tax Incentives in End-of-the-Yea...,Press Release,C000174,116
...,...,...,...,...,...,...
34835,https://www.cantwell.senate.gov/news/press-rel...,2022-08-17,Cantwell Touts New Electric Lummi Island Ferry...,Press Release,C000127,117
34836,https://www.cantwell.senate.gov/news/press-rel...,2022-08-17,"Cantwell, Murray Visit SW WA Semiconductor Man...",Press Release,C000127,117
34837,https://www.cantwell.senate.gov/news/press-rel...,2022-08-16,Cantwell Hails Bothell Way Project Made Possib...,Press Release,C000127,117
34838,https://www.cantwell.senate.gov/news/press-rel...,2022-08-11,"Cantwell, DOE Secretary Granholm Tour PNNL and...",Press Release,C000127,117


In [15]:
member_statements = pd.concat([current_statements_prod, current_statements_cache])
member_statements_ids = member_statements[['member_id', 'congress']].drop_duplicates()

member_statements_ids.rename(columns={'member_id': 'id'}, inplace=True)

In [16]:
merged_df = new_ids.merge(member_statements_ids, on=['congress', 'id'], how='outer', indicator=True)

# Filter out rows that are found in both DataFrames
unique_to_df1 = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])

unique_to_df1

,id,congress
20,L000594,116
57,W000802,117
58,S001217,117
59,H000273,117
60,S001198,117
61,C001035,117
62,B001236,117
63,M000355,117
64,H000601,117
65,P000595,117


In [17]:
url_list = []
date_list = []
title_list = []
type_list = []
member_list = []
congress_list = []

iteration_count = 0
save_interval = 1

for index, row in unique_to_df1.iterrows():
    iteration_count +=1
    member_id = row['id']
    congress = row['congress']
    
    url = f"https://api.propublica.org/congress/v1/members/{member_id}/statements/{congress}.json"
    response = requests.get(url, headers=headers)
    if 'results' in response.json().keys():
        if response.json()['num_results'] >0:
            total = response.json()['num_results']
            count = 20
            pages = int(np.ceil(total/count))
            offset = 0
            
            for i in range(1, pages+1):
                url = f"https://api.propublica.org/congress/v1/members/{member_id}/statements/{congress}.json?page={offset}"
                response = requests.get(url, headers=headers)
                results = response.json()['results']
                offset += 20
                for result in results:
                    url_list.append(result['url'])
                    date_list.append(result['date'])
                    title_list.append(result['title'])
                    type_list.append(result['statement_type'])
                    member_list.append(member_id)
                    congress_list.append(congress)
        else:
            pass
    else:
        pass

    if iteration_count % save_interval == 0:
        print(f"Saving {iteration_count} rows")
        df_temp = pd.DataFrame({'url': url_list, 
                                'date': date_list, 
                                'title': title_list, 
                                'type': type_list, 
                                'member_id': member_list, 
                                'congress': congress_list})
        df_temp.to_csv('member_statements.csv', index=False)
    

final_statements = pd.DataFrame({'url': url_list, 
                                  'date': date_list, 
                                  'title': title_list, 
                                  'type': type_list, 
                                  'member_id': member_list, 
                                  'congress': congress_list})

Saving 1 rows
Saving 2 rows
Saving 3 rows
Saving 4 rows
Saving 5 rows
Saving 6 rows
Saving 7 rows
Saving 8 rows
Saving 9 rows
Saving 10 rows
Saving 11 rows
Saving 12 rows
Saving 13 rows
Saving 14 rows
Saving 15 rows
Saving 16 rows
Saving 17 rows
Saving 18 rows
Saving 19 rows
Saving 20 rows
Saving 21 rows
Saving 22 rows
Saving 23 rows
Saving 24 rows
Saving 25 rows
Saving 26 rows
Saving 27 rows
Saving 28 rows
Saving 29 rows
Saving 30 rows
Saving 31 rows
Saving 32 rows
Saving 33 rows
Saving 34 rows
Saving 35 rows
Saving 36 rows
Saving 37 rows
Saving 38 rows
Saving 39 rows
Saving 40 rows
Saving 41 rows
Saving 42 rows
Saving 43 rows
Saving 44 rows
Saving 45 rows
Saving 46 rows
Saving 47 rows


In [18]:
member_statements

,url,date,title,type,member_id,congress
0,https://susielee.house.gov/media/press-release...,2020-12-03,Rep. Lee Announces Nearly $1.4 Million Grant t...,Press Release,L000590,116
1,https://susielee.house.gov/media/press-release...,2020-12-01,"Problem Solvers Caucus, Senate Partners Unveil...",Press Release,L000590,116
2,https://susielee.house.gov/media/press-release...,2020-11-20,Rep. Susie Lee Helps Pass National Apprentices...,Press Release,L000590,116
3,https://susielee.house.gov/media/press-release...,2020-11-18,Rep. Susie Lee Elected to Serve as Regional Re...,Press Release,L000590,116
4,https://susielee.house.gov/media/press-release...,2020-12-27,"Rep. Lee Statement, President Trump Finally Si...",Press Release,L000590,116
...,...,...,...,...,...,...
34835,https://www.cantwell.senate.gov/news/press-rel...,2022-08-17,Cantwell Touts New Electric Lummi Island Ferry...,Press Release,C000127,117
34836,https://www.cantwell.senate.gov/news/press-rel...,2022-08-17,"Cantwell, Murray Visit SW WA Semiconductor Man...",Press Release,C000127,117
34837,https://www.cantwell.senate.gov/news/press-rel...,2022-08-16,Cantwell Hails Bothell Way Project Made Possib...,Press Release,C000127,117
34838,https://www.cantwell.senate.gov/news/press-rel...,2022-08-11,"Cantwell, DOE Secretary Granholm Tour PNNL and...",Press Release,C000127,117


In [19]:
final_statements

,url,date,title,type,member_id,congress
0,http://www.whitehouse.senate.gov/news/videos/w...,2022-09-15,Sen. Whitehouse on Sea Level Rise in Rhode Isl...,Press Release,W000802,117
1,http://www.whitehouse.senate.gov/news/videos/w...,2022-09-15,Sen. Whitehouse Joins The Last Word to Discuss...,Press Release,W000802,117
2,http://www.whitehouse.senate.gov/news/release/...,2022-09-15,RI Delegation Announces Major $82.5 Million Gr...,Press Release,W000802,117
3,http://www.whitehouse.senate.gov/news/release/...,2022-09-15,"Whitehouse, Grassley Lead Senate Caucus in Iss...",Press Release,W000802,117
4,http://www.whitehouse.senate.gov/news/videos/w...,2022-09-14,Sen. Whitehouse and Lawrence O'Donnell Discuss...,Press Release,W000802,117
...,...,...,...,...,...,...
11615,https://www.coons.senate.gov/news/press-releas...,2023-10-27,"Senators Coons, Scott, colleagues call on Sena...",Press Release,C001088,118
11616,https://www.coons.senate.gov/news/press-releas...,2023-10-25,"ICYMI: Following debate, Senators Coons, Rubio...",Press Release,C001088,118
11617,https://www.coons.senate.gov/news/press-releas...,2023-10-24,"Senator Coons, colleagues write bipartisan let...",Press Release,C001088,118
11618,https://www.coons.senate.gov/news/press-releas...,2023-10-23,Senator Coons statement on congressional deleg...,Press Release,C001088,118


In [21]:
final_df = pd.concat([member_statements, final_statements])

final_df.to_csv('../prod_data/member_statements.csv', index=False)